In [ ]:
!pip install selenium

In [ ]:
!pip install webdriver_manager

In [ ]:
!pip install bs4

In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())

In [ ]:
driver.get("https://edugate.ksu.edu.sa/ksu/ui/guest/timetable/index/mainScheduleTreeCoursesIndex.faces")

In [ ]:
driver.find_element_by_id('myForm:timetableTreeId').click()

In [ ]:
def has_setIndex(tag):
    return tag.has_attr('href') and 'setIndex' in tag['href']

In [ ]:
source = driver.page_source
sp = BeautifulSoup(source,'lxml')
tree = sp.find('div',{"class":"dtree"})

majors_dict = {}
for i in tree.find_all(has_setIndex):
    start = i['href'].find("'") + 1
    end = i['href'].rfind("'")
    majors_dict[int(i['href'][start:end])] = i.text 

In [ ]:
print("%5s" % "id:","Major Name")
for i, major in majors_dict.items():
    print("%5d" %i, ':', major)

In [ ]:
def select_major(mId):
    x = str(mId)
    driver.execute_script(f"setIndex('{x[2:]}_{x[-2]}_{x[-1]}')")

In [ ]:
mId = int(input("Enter major id from the above table:"))
if mId not in majors_dict:
    print("Wrong ID, try again!")

In [ ]:
select_major(mId)

In [ ]:
driver.find_element_by_xpath('//*[@id="myForm"]/table[4]/tbody/tr[1]/td/table/tbody/tr/td/table/tbody/tr[5]/td[6]/a[2]/table/tbody/tr/td[1]/img').click()

In [ ]:
content = driver.page_source
soup = BeautifulSoup(content,features='lxml')
rt= soup.find(id="myForm:timetable")
# rt

In [ ]:
trs = rt.find_all('tr')
print(len(trs[1:]))

In [ ]:
def press_details(index=1):
    details_selector = '#myForm\:timetable > tbody > tr:nth-child({}) > td:nth-child(8) > a'
    details_button = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,details_selector.format(index))))
    details_button.click()

In [ ]:
def press_details_js(index=0):
    driver.execute_script(f"showToolTipDilaog(event,'{index}');")

In [ ]:
def get_times():
#     times = driver.find_element_by_id('sectionDiv').find_elements_by_tag_name('nobr')
    times = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.ID,'sectionDiv'))).find_elements_by_tag_name('nobr')
    if not times:
        return None
    n=len(times)//3
    return [times[i*3].text for i in range(n)], [times[1+i*3].text for i in range(n)] , [times[2+i*3].text for i in range(n)]

In [ ]:
def get_final_info():
    return driver.find_element_by_id('examDayDiv').text,driver.find_element_by_id('examTimeDiv').text,driver.find_element_by_id('examDateDiv').text

In [ ]:
def get_instructor():
    return driver.find_element_by_id('instructorDiv').text

In [ ]:
def close():
    driver.find_element_by_xpath('//*[@id="dlg"]/div[1]/a/span').click()

In [ ]:
ctrs = trs[1:]
len(ctrs)

In [ ]:
# NOTE: the following method does not collect courses where the activity is "تدريب" or "مشروع".
# If you want to include thenm
res = []
for x, i in enumerate(ctrs):
    tds = i.find_all('td')
    if 'مشروع' in tds[3].text or 'تدريب' in tds[3].text: continue
    tmp = {
        'cid': tds[0].text,
        'cname': tds[1].text,
        'section': tds[2].text,
        'activity': tds[3].text,
        'chours': tds[4].text,
        'gender': tds[5].text,
        'status': tds[6].text
    }
    press_details_js(x)
    driver.implicitly_wait(5)
    times = get_times()
    tmp['days'],tmp['time'],tmp['location']=(times) if times else ('','','')
    final_info = get_final_info()
    tmp['fexamday'],tmp['fexamtime'],tmp['fexamdate']=final_info

    tmp['instructor'] = get_instructor()
    driver.implicitly_wait(5)
    close()
    driver.implicitly_wait(5)
    res.append(tmp)

In [ ]:
len(res)

In [ ]:
import csv
keys = res[0].keys()

with open(f'{majors_dict[mId]}.csv', 'w',encoding='utf-8-sig', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(res)